# Caesar code deciphering with keras

### Step 1. Data download and exploration

In [2]:
import helper

codes = helper.load_data('cipher.txt')
plaintext = helper.load_data('plaintext.txt')

In [3]:
print('Examples of encoded messages: \n', *codes[:3], sep = "\n")

Examples of encoded messages: 

YMJ QNRJ NX MJW QJFXY QNPJI KWZNY , GZY YMJ GFSFSF NX RD QJFXY QNPJI .
MJ XFB F TQI DJQQTB YWZHP .
NSINF NX WFNSD IZWNSL OZSJ , FSI NY NX XTRJYNRJX BFWR NS STAJRGJW .


In [4]:
print('Examples of plaintext messages: \n', *plaintext[:3], sep = "\n")

Examples of plaintext messages: 

THE LIME IS HER LEAST LIKED FRUIT , BUT THE BANANA IS MY LEAST LIKED .
HE SAW A OLD YELLOW TRUCK .
INDIA IS RAINY DURING JUNE , AND IT IS SOMETIMES WARM IN NOVEMBER .


### Step 2. Preprocessing

In [6]:
from keras.preprocessing.text import Tokenizer

def tokenize(x):
    x_tk = Tokenizer(char_level = True)
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk
  
  
# Tokenization example
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)

print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{' ': 1, 'e': 2, 'o': 3, 't': 4, 'i': 5, 's': 6, 'h': 7, 'r': 8, 'y': 9, 'u': 10, 'c': 11, 'n': 12, 'a': 13, 'p': 14, '.': 15, 'q': 16, 'k': 17, 'b': 18, 'w': 19, 'f': 20, 'x': 21, 'j': 22, 'm': 23, 'v': 24, 'l': 25, 'z': 26, 'd': 27, 'g': 28, ',': 29}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [4, 7, 2, 1, 16, 10, 5, 11, 17, 1, 18, 8, 3, 19, 12, 1, 20, 3, 21, 1, 22, 10, 23, 14, 6, 1, 3, 24, 2, 8, 1, 4, 7, 2, 1, 25, 13, 26, 9, 1, 27, 3, 28, 1, 15]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [18, 9, 1, 22, 3, 24, 2, 1, 29, 1, 23, 9, 1, 16, 10, 5, 11, 17, 1, 6, 4, 10, 27, 9, 1, 3, 20, 1, 25, 2, 21, 5, 11, 3, 28, 8, 13, 14, 7, 9, 1, 19, 3, 12, 1, 13, 1, 14, 8, 5, 26, 2, 1, 15]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [4, 7, 5, 6, 1, 5, 6, 1, 13, 1, 6, 7, 3, 8, 4, 1, 6, 2, 12, 4, 2, 12, 11, 2, 1, 15]


In [10]:
import numpy as np
from tensorflow.keras.utils import pad_sequences

def pad(x, length=None):
    if length is None:
        length = max(len(i) for i in x)
    padded_sequences = pad_sequences(x, maxlen=length, padding = 'post')
    
    return padded_sequences

# Padded sequences example
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 4  7  2  1 16 10  5 11 17  1 18  8  3 19 12  1 20  3 21  1 22 10 23 14
  6  1  3 24  2  8  1  4  7  2  1 25 13 26  9  1 27  3 28  1 15]
  Output: [ 4  7  2  1 16 10  5 11 17  1 18  8  3 19 12  1 20  3 21  1 22 10 23 14
  6  1  3 24  2  8  1  4  7  2  1 25 13 26  9  1 27  3 28  1 15  0  0  0
  0  0  0  0  0  0]
Sequence 2 in x
  Input:  [18  9  1 22  3 24  2  1 29  1 23  9  1 16 10  5 11 17  1  6  4 10 27  9
  1  3 20  1 25  2 21  5 11  3 28  8 13 14  7  9  1 19  3 12  1 13  1 14
  8  5 26  2  1 15]
  Output: [18  9  1 22  3 24  2  1 29  1 23  9  1 16 10  5 11 17  1  6  4 10 27  9
  1  3 20  1 25  2 21  5 11  3 28  8 13 14  7  9  1 19  3 12  1 13  1 14
  8  5 26  2  1 15]
Sequence 3 in x
  Input:  [ 4  7  5  6  1  5  6  1 13  1  6  7  3  8  4  1  6  2 12  4  2 12 11  2
  1 15]
  Output: [ 4  7  5  6  1  5  6  1 13  1  6  7  3  8  4  1  6  2 12  4  2 12 11  2
  1 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0]


In [12]:
# Preprocessing pipeline

def preprocess(x, y):

    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer =\
    preprocess(codes, plaintext)

print('Data Preprocessed')

Data Preprocessed


### Step 3. Sample model

In [38]:
from keras.layers import GRU, Input, Dense, TimeDistributed, Lambda, Embedding
from keras.models import Model, Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras import backend as K
from keras.utils import to_categorical

# Define a baseline model

def simple_model(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    
    embedding_dim = 256
    
    model = Sequential([
        Embedding(input_dim = output_sequence_length, output_dim = embedding_dim),
        GRU(64, return_sequences = True, input_shape = (101,1)),
        Dense(code_vocab_size, activation = 'softmax')
    ])
    
    model.compile(optimizer = Adam(1e-3), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [39]:
# Prepare and reshape the X_train and y_train

tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1)).reshape(10001, 101)

preproc_plaintext_sentences = preproc_plaintext_sentences.reshape(10001, 101)
tmp_x.shape

(10001, 101)

In [40]:
# Define tje model and print the summary

simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

simple_rnn_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 256)         25856     
                                                                 
 gru_8 (GRU)                 (None, None, 64)          61824     
                                                                 
 dense_8 (Dense)             (None, None, 32)          2080      
                                                                 
Total params: 89,760
Trainable params: 89,760
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Fit the model to the train data
simple_rnn_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=3, validation_split=0.2)

Epoch 1/3
250/250 [==============================] - 5s 22ms/step - loss: 0.0060 - accuracy: 0.9999 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 2/3
250/250 [==============================] - 5s 22ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 3/3
250/250 [==============================] - 5s 22ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 8.4304e-04 - val_accuracy: 1.0000
